In [32]:
from mongoengine import *
import os
import pandas as pd
import numpy as np
import re

使用** mongoengine **  
[connect](http://docs.mongoengine.org/guide/connecting.html#guide-connecting)进行数据库连接

In [33]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## 1 读取文件

### 读取方式1：使用pandas

In [34]:
targetfilename=r"CH2017BST.txt"
targetpath=r"/Users/casablanca/03project/typhoonSearchSys/demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [35]:
fullname

'/Users/casablanca/03project/typhoonSearchSys/demo_data/CH2017BST.txt'

不使用pandas的read_table

In [36]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\s+',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


In [37]:
data.head(3)

,0,1,2,3,4,5,6,7,8
0,66666,0,25,0,1,0,6.0,(nameless),20180501.0
1,2017041406,0,109,1302,1010,10,NaN,NaN,NaN
2,2017041412,0,107,1290,1010,10,NaN,NaN,NaN


In [38]:
len(data)

857

In [39]:
data.iloc[0]

0          66666
1              0
2             25
3              0
4              1
5              0
6              6
7     (nameless)
8    2.01805e+07
Name: 0, dtype: object

--------

## 2 判断标志位的方式（必须同时满足），并获取标志位所在位置的数组（mark_indexs）

1-第一位是否为66666

In [40]:
data.iloc[0][0]==66666

True

2- 第7位是否不为Nan

In [41]:
data.iloc[0][7]!=np.nan

True

In [42]:
data.iloc[0][0]==66666 and data.iloc[0][7]!=np.nan

True

### 写成方法：

In [55]:
def checkMark(index):
    '''
        判断是否为标志位
    '''
    return data.iloc[index][0]==66666 and data.iloc[index][7]!=np.nan

找到标志位所在的位置

In [56]:
mark_indexs=[]
for i in range(len(data)):
#     print(i)
    if checkMark(i):
        mark_indexs.append(i)

In [57]:
mark_indexs[:5]

[0, 26, 54, 76, 98]

--------

## 3 获取观测值并写入mongoDB（暂时先写在list中）

In [63]:
mark_indexs[:3]

[0, 26, 54]

In [61]:
mark_indexs[1:3]

[26, 54]

In [66]:
# 保存起止数组的数组
list_startend=[]
index=0
for val in mark_indexs:
#     list_startend.append(mark_indexs[index:2])
#     print(mark_indexs[index:index+2])
    list_startend.append(mark_indexs[index:index+2])
    index=index+1
#     print(index)

In [67]:
list_startend[0:3]

[[0, 26], [26, 54], [54, 76]]

In [68]:
list_realdata=[]

** 获取第1行-25行，0-6列的数据 **

In [82]:
data.iloc[1:25,0:6].head()

,0,1,2,3,4,5
1,2017041406,0,109,1302,1010,10
2,2017041412,0,107,1290,1010,10
3,2017041418,0,107,1280,1010,10
4,2017041500,1,109,1272,1008,13
5,2017041506,1,111,1263,1008,13


In [83]:
list_realdata.append(data.iloc[1:25,0:6])

In [85]:
# list_realdata

----

## 4 使用mongoEngine

In [86]:
from mongoengine import *

In [87]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [ ]:
class typh

定义 mongodb中的存储对象（ORM）

In [118]:
class Point(EmbeddedDocument):
    '''
        点（经纬度）
    '''
    lat=FloatField()
    lon=FloatField()
#     class my_metaclass:
#         allow_inheritance=True

class GeoTyphoonRealData(Document):
    '''
        支持geojson的存储至mongodb的model
    '''

    code=StringField(max_length=10)
    date=DateTimeField()
    bp=FloatField()
    wsm=FloatField()
    # latlon=models.ForeignKey(Point,on_delete=models.CASCADE)
    latlon=EmbeddedDocumentField('Point')
#     meta = {'db_alias': 'Typhoon_geotyphoonrealdata'}
#     meta = {'collection': 'geotyphoonrealdata'}
    meta = {'collection': 'Typhoon_geotyphoonrealdata'}
    # object=MongoDBManager()

In [119]:
import datetime

In [120]:
typhoon_temp=GeoTyphoonRealData(code="",
                                date=datetime.datetime.now(),
                                bp=1010.9,
                                wsm=5.4,
                                latlon=Point(lat=18.2,lon=97.9))

In [121]:
typhoon_temp.save()

<GeoTyphoonRealData: GeoTyphoonRealData object>

** 提示错误如下 **:  
NotUniqueError: Tried to save duplicate unique keys (E11000 duplicate key error collection: typhoon.Typhoon_geotyphoonrealdata index: __primary_key__ dup key: { : null })